In [2]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import pandas_ta as ta
import backtrader as bt
import MetaTrader5 as mt5
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sesto.constants import CURRENCY_PAIRS
from sesto.indicators import SMA, BB, RSI
import sesto.metatrader.data as mtd

MetaTrader 5 initialized successfully.


In [3]:
PAIRS = CURRENCY_PAIRS
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 3000
RSI_PERIOD = 14

# Configuration Variables
RSI_LOWER = 20
RSI_UPPER =80
INITIAL_CAPITAL = 10000
LEVERAGE = 10  # Leverage factor (e.g., 10x)
TP_SL_RATIO = 2  # Ratio between TP and SL (e.g., 2:1)
SPREAD_PERCENT = 0.012  # Overall spread between entry and TP/SL (e.g., 3%)
TRANSACTION_COST = 0.001  # Transaction cost as a percentage of the trade value (e.g., 0.1%)
CAPITAL_ALLOCATION_PER_TRADE = 0.1  # Allocate 10% of capital per trade

In [4]:
mtd.fill_data(PAIRS, TIMEFRAME, BARS)

Fetched data for pair: USDJPY
Fetched data for pair: EURUSD
Fetched data for pair: GBPUSD
Fetched data for pair: EURGBP
Fetched data for pair: CADCHF
Fetched data for pair: EURJPY
Fetched data for pair: AUDUSD
Fetched data for pair: USDCNH
Fetched data for pair: EURCHF
Fetched data for pair: NZDUSD


In [5]:
for symbol, df in mtd.data.items():
    RSI(df, RSI_PERIOD)

mtd.data['AUDUSD']

,time,open,high,low,close,tick_volume,spread,real_volume,rsi-14
13,2024-03-28 01:00:00,0.65199,0.65231,0.65181,0.65227,902,11,0,45.229008
14,2024-03-28 02:00:00,0.65227,0.65282,0.65151,0.65254,1718,3,0,47.912886
15,2024-03-28 03:00:00,0.65254,0.65302,0.65211,0.65280,2207,3,0,55.555556
16,2024-03-28 04:00:00,0.65280,0.65406,0.65275,0.65352,1852,4,0,67.161410
17,2024-03-28 05:00:00,0.65352,0.65384,0.65310,0.65315,920,4,0,59.082218
...,...,...,...,...,...,...,...,...,...
2995,2024-09-18 19:00:00,0.67573,0.67663,0.67560,0.67658,1292,8,0,49.834620
2996,2024-09-18 20:00:00,0.67658,0.67703,0.67617,0.67655,1476,8,0,52.073733
2997,2024-09-18 21:00:00,0.67900,0.68199,0.67807,0.67859,5818,6,0,68.404588
2998,2024-09-18 22:00:00,0.67855,0.67855,0.67418,0.67479,4865,6,0,45.707472


In [6]:


# Initialize an empty list to store trade details
trades = []

# Iterate through each symbol's data
for symbol, df in mtd.data.items():
    capital = INITIAL_CAPITAL
    open_positions = []

    for index, row in df.iterrows():
        rsi = row[f'rsi-{RSI_PERIOD}']
        close_price = row['close']
        trade_time = row['time']

        # Check for entry signals
        if rsi < RSI_LOWER or rsi > RSI_UPPER:
            if (rsi < RSI_LOWER and not any(pos['position'] == 'long' for pos in open_positions)) or \
               (rsi > RSI_UPPER and not any(pos['position'] == 'short' for pos in open_positions)):

                position_type = 'long' if rsi < RSI_LOWER else 'short'
                entry_price = close_price
                allocated_capital = capital * CAPITAL_ALLOCATION_PER_TRADE
                spread = entry_price * SPREAD_PERCENT
                tp = entry_price + spread if position_type == 'long' else entry_price - spread
                sl = entry_price - (spread / TP_SL_RATIO) if position_type == 'long' else entry_price + (spread / TP_SL_RATIO)
                
                # Calculate transaction cost when opening a position
                transaction_cost = TRANSACTION_COST * LEVERAGE * allocated_capital
                allocated_capital -= transaction_cost
                capital -= allocated_capital
                
                open_positions.append({
                    'symbol': symbol,
                    'entry_time': trade_time,
                    'entry_price': entry_price,
                    'sl': sl,
                    'tp': tp,
                    'position': position_type,
                    'allocated_capital': allocated_capital,
                    'transaction_cost': transaction_cost
                })

        # Check for exit signals
        for position in open_positions.copy():
            if position['position'] == 'long' and (rsi > RSI_UPPER or close_price >= position['tp'] or close_price <= position['sl']):
                profit = (close_price - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                capital += profit - position['transaction_cost']
                max_drawdown = (position['entry_price'] - min(df.loc[index:, 'close'])) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                
                trades.append({
                    'symbol': position['symbol'],
                    'entry_time': position['entry_time'],
                    'exit_time': trade_time,
                    'entry_price': position['entry_price'],
                    'close_price': close_price,
                    'pnl': profit - position['transaction_cost'],
                    'max_drawdown': max_drawdown,
                    'tp': position['tp'],
                    'sl': position['sl'],
                    'position': position['position']
                })
                open_positions.remove(position)
            
            elif position['position'] == 'short' and (rsi < RSI_LOWER or close_price <= position['tp'] or close_price >= position['sl']):
                profit = (position['entry_price'] - close_price) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                capital += profit - position['transaction_cost']
                max_drawdown = (max(df.loc[index:, 'close']) - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
                
                trades.append({
                    'symbol': position['symbol'],
                    'entry_time': position['entry_time'],
                    'exit_time': trade_time,
                    'entry_price': position['entry_price'],
                    'close_price': close_price,
                    'pnl': profit - position['transaction_cost'],
                    'max_drawdown': max_drawdown,
                    'tp': position['tp'],
                    'sl': position['sl'],
                    'position': position['position']
                })
                open_positions.remove(position)

    # Ensure that any open positions are closed at the final market price
    final_row = df.iloc[-1]
    final_price = final_row['close']
    final_time = final_row['time']

    for position in open_positions:
        if position['position'] == 'long':
            profit = (final_price - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
        elif position['position'] == 'short':
            profit = (position['entry_price'] - final_price) * LEVERAGE * position['allocated_capital'] / position['entry_price']

        # Calculate transaction cost when closing a position
        transaction_cost = TRANSACTION_COST * LEVERAGE * position['allocated_capital']
        capital += profit - transaction_cost

        max_drawdown = (position['entry_price'] - min(df['close'])) * LEVERAGE * position['allocated_capital'] / position['entry_price'] if position['position'] == 'long' else (max(df['close']) - position['entry_price']) * LEVERAGE * position['allocated_capital'] / position['entry_price']
        
        trades.append({
            'symbol': position['symbol'],
            'entry_time': position['entry_time'],
            'exit_time': final_time,
            'entry_price': position['entry_price'],
            'close_price': final_price,
            'pnl': profit - transaction_cost,
            'max_drawdown': max_drawdown,
            'tp': position['tp'],
            'sl': position['sl'],
            'position': position['position']
        })

# Convert trades to a DataFrame for analysis
trades_df = pd.DataFrame(trades)
trades_df['capital'] = INITIAL_CAPITAL + trades_df['pnl'].cumsum()

trades_df

,symbol,entry_time,exit_time,entry_price,close_price,pnl,max_drawdown,tp,sl,position,capital
0,USDJPY,2024-04-01 18:00:00,2024-04-10 16:00:00,151.72300,152.67000,-71.792213,666.598999,149.902324,152.633338,short,9928.207787
1,USDJPY,2024-04-10 17:00:00,2024-04-15 05:00:00,152.77100,153.70700,-63.153346,531.030327,150.937748,153.687626,short,9865.054441
2,USDJPY,2024-04-15 06:00:00,2024-04-16 14:00:00,153.67100,154.61800,-56.737338,425.598282,151.826948,154.593026,short,9808.317102
3,USDJPY,2024-04-18 19:00:00,2024-04-25 07:00:00,154.61800,155.58800,-51.502404,334.803644,152.762584,155.545708,short,9756.814698
4,USDJPY,2024-04-25 08:00:00,2024-04-26 10:00:00,155.65900,156.70100,-48.690435,254.976606,153.791092,156.592954,short,9708.124264
...,...,...,...,...,...,...,...,...,...,...,...
475,NZDUSD,2024-08-29 10:00:00,2024-09-03 06:00:00,0.62817,0.61988,0.972359,-0.589348,0.620632,0.631939,short,9361.791388
476,NZDUSD,2024-09-03 07:00:00,2024-09-04 17:00:00,0.61900,0.62072,0.128842,0.885035,0.626428,0.615286,long,9361.920230
477,NZDUSD,2024-09-04 17:00:00,2024-09-09 13:00:00,0.62072,0.61309,0.740537,0.256962,0.613271,0.624444,short,9362.660767
478,NZDUSD,2024-09-12 07:00:00,2024-09-13 03:00:00,0.61507,0.61906,-0.449836,0.788214,0.607689,0.618760,short,9362.210931


In [7]:
from sesto.Performance import performance
performance(trades_df, INITIAL_CAPITAL)

ModuleNotFoundError: No module named 'sesto.Performance'

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_trades_for_all_symbols(trades_df, shared_data):
    # Define colors for different trade actions
    entry_color = '#2ca02c'  # Green for entry
    exit_color = '#d62728'   # Red for exit
    tp_color = '#1f77b4'     # Blue for Take Profit
    sl_color = '#ff7f0e'     # Orange for Stop Loss

    # Iterate through each symbol's data
    for symbol, price_data in shared_data.items():
        # Ensure time columns are in datetime format
        price_data['time'] = pd.to_datetime(price_data['time'])

        # Initialize figure with subplots (Price Chart and RSI)
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                            row_heights=[0.7, 0.3], vertical_spacing=0.1,
                            subplot_titles=(f'{symbol} Price Chart', 'RSI'))

        # Plot the price chart (legend appears only once)
        fig.add_trace(go.Scatter(x=price_data['time'], y=price_data['close'], 
                                 mode='lines', name=f'{symbol} Close Price', 
                                 line=dict(color='#1f77b4', width=2),
                                 showlegend=True), row=1, col=1)

        # Plot RSI if available (legend appears only once)
        if f'rsi-{RSI_PERIOD}' in price_data.columns:
            fig.add_trace(go.Scatter(x=price_data['time'], y=price_data[f'rsi-{RSI_PERIOD}'], 
                                     mode='lines', name='RSI', 
                                     line=dict(color='#ff7f0e', dash='dash'),
                                     showlegend=True), row=2, col=1)
            # Add RSI thresholds (these won't be in the legend)
            fig.add_hline(y=30, line=dict(color='red', dash='dash'), row=2, col=1)
            fig.add_hline(y=70, line=dict(color='green', dash='dash'), row=2, col=1)

        # Filter trades for the current symbol
        symbol_trades = trades_df[trades_df['symbol'] == symbol].copy()
        symbol_trades['entry_time'] = pd.to_datetime(symbol_trades['entry_time'])
        symbol_trades['exit_time'] = pd.to_datetime(symbol_trades['exit_time'])

        # Track whether TP and SL have been added to the legend
        tp_in_legend = False
        sl_in_legend = False

        # Plot each trade on the price chart
        for i, trade in symbol_trades.iterrows():
            entry_time = trade['entry_time']
            exit_time = trade['exit_time']
            entry_price = trade['entry_price']
            tp = trade['tp']
            sl = trade['sl']
            close_price = trade['close_price']
            pnl = trade['pnl']

            # Plot trade entry/exit as line with markers (only one legend entry for all trades)
            fig.add_trace(go.Scatter(
                x=[entry_time, exit_time], 
                y=[entry_price, close_price], 
                mode='lines+markers', 
                marker=dict(color=entry_color if pnl > 0 else exit_color, size=8),
                line=dict(color=entry_color if pnl > 0 else exit_color, width=2),
                name=f'Trade {"Profit" if pnl > 0 else "Loss"}' if i == 0 else '',
                showlegend=(i == 0)  # Show legend only for the first trade
            ), row=1, col=1)

            # Plot TP and SL as narrow lines between entry and exit
            fig.add_trace(go.Scatter(
                x=[entry_time, exit_time], 
                y=[tp, tp], 
                mode='lines', 
                line=dict(color=tp_color, dash='dot', width=2),
                name='Take Profit' if not tp_in_legend else '',
                showlegend=not tp_in_legend  # Show legend only for the first TP
            ), row=1, col=1)
            tp_in_legend = True

            fig.add_trace(go.Scatter(
                x=[entry_time, exit_time], 
                y=[sl, sl], 
                mode='lines', 
                line=dict(color=sl_color, dash='dot', width=2),
                name='Stop Loss' if not sl_in_legend else '',
                showlegend=not sl_in_legend  # Show legend only for the first SL
            ), row=1, col=1)
            sl_in_legend = True

        # Update layout with titles and axis labels
        fig.update_layout(
            title=f'{symbol} Price and RSI with Trades',
            xaxis_title='Date',
            yaxis_title='Price',
            height=700,
            showlegend=True,
            template="plotly_dark"
        )

        fig.update_yaxes(title_text='RSI', row=2, col=1)
        fig.update_xaxes(rangeslider_visible=False)
        
        # Show the interactive plot
        fig.show()

# Example usage (commented out because it requires data):
plot_trades_for_all_symbols(trades_df, mtd.data)


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Calculate performance metrics for each symbol
performance_summary = trades_df.groupby('symbol').agg(
    total_pnl=('pnl', 'sum'),
    max_drawdown=('max_drawdown', 'max'),
    trade_count=('symbol', 'size'),
    win_ratio=('pnl', lambda x: (x > 0).sum() / len(x))  # Win ratio = # of winning trades / total trades
).reset_index()

# Initialize a Plotly figure
fig = go.Figure()

# Create a bar chart for each metric
fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['total_pnl'],
    name='Total PnL',
))

fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['max_drawdown'],
    name='Max Drawdown',
))

fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['trade_count'],
    name='Trade Count',
))

fig.add_trace(go.Bar(
    x=performance_summary['symbol'],
    y=performance_summary['win_ratio'] * 100,  # Convert win ratio to percentage
    name='Win Ratio (%)',
))

# Update layout for better readability
fig.update_layout(
    title='Performance Metrics per Symbol',
    barmode='group',  # Group bars for better comparison
    xaxis_title='Symbol',
    yaxis_title='Value',
    legend_title='Metrics',
    height=600,
    template='plotly_dark'
)

# Show the interactive plot
fig.show()
